<a href="https://colab.research.google.com/github/JeyTee1988/LLM-Schulung/blob/main/llm_schulung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Wichtig**: Wenn Du dieses Notebook in Google Colab ausführst, wähle zuerst unter Laufzeit > Laufzeittyp ändern eine GPU aus

Zunächst installieren wir die benötigte Library Huggingface Transformer

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00


# Klassifizierung von Text

Wir laden eine Pipeline mit einem vortrainierten Sentiment-Modell. Das Modell kann explizit angegeben werden, ansonsten wird ein Standard-Modell genommen.

In [2]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Wir können das Modell nutzen, um die Stimmung eines Textes zu bestimmen. Hierbei wird angegeben, ob der Text positiv oder negativ ist und wie sicher das Modell ist bei der Bewertung.

In [3]:
classifier(["We are happy to have you participate in our modest LLM workshop!", "Sorry that we had to cap the participant limit to 20"])

[{'label': 'POSITIVE', 'score': 0.9998558759689331},
 {'label': 'NEGATIVE', 'score': 0.9991243481636047}]

Dieses Modell wurde nur mit englischen Texten und scheinbar nicht mit Emoji trainiert:

In [4]:
classifier(["Was ein schöner Tag für eine Schulung", "🤗", "😠"])

[{'label': 'NEGATIVE', 'score': 0.9137823581695557},
 {'label': 'NEGATIVE', 'score': 0.6970564126968384},
 {'label': 'NEGATIVE', 'score': 0.6970564126968384}]

# Textgenerierung

Wir laden das vortrainierte distilgpt2 Modell und nutzen es für Textgenerierung

In [5]:
from transformers import pipeline

pipe = pipeline("text-generation", model="distilgpt2")

Wir können das Modell nutzen, um Texte zu vervollständigen.

In [6]:
from transformers import set_seed
set_seed(42)
pipe("Hello, I’m a language model", max_length=20, num_return_sequences=5)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hello, I’m a language model but what I do in that role is to make everything'},
 {'generated_text': 'Hello, I’m a language model.” You’ll know the real name'},
 {'generated_text': 'Hello, I’m a language model’s\xadself, and I’m'},
 {'generated_text': 'Hello, I’m a language model, and I really wanted to make a nice API,'},
 {'generated_text': 'Hello, I’m a language model. I’think in any language. I�'}]

# Tokenizer

Ein Tokenizer in einem Language Learning Model (LLM) dient dazu, Text in kleinere Einheiten, sogenannte Tokens, zu zerlegen. Diese Tokens können einzelne Wörter, Phrasen oder sogar einzelne Buchstaben sein.

Der Tokenizer ist wichtig, weil ein LLM wie ein neuronales Netzwerk arbeitet, das auf Zahlen, nicht auf Wörtern, basiert. Es kann nicht direkt Text verstehen oder interpretieren. Stattdessen muss es den Text in eine numerische Form umwandeln, die es verarbeiten kann.

Dieser Prozess des Zerlegens und Umwandeln wird durch den Tokenizer durchgeführt. Er nimmt den ursprünglichen Text, teilt ihn in Tokens und wandelt diese dann in Zahlen um, die das Modell verarbeiten kann.

https://platform.openai.com/tokenizer


In [7]:
tokenizer = pipe.tokenizer
model = pipe.model

text = "Hello, I’m a language model"
tokenizer.tokenize(text)

['Hello', ',', 'ĠI', 'âĢ', 'Ļ', 'm', 'Ġa', 'Ġlanguage', 'Ġmodel']

In [8]:
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input.input_ids

tensor([[15496,    11,   314,   447,   247,    76,   257,  3303,  2746]])

Der Tokenizer enthält ein Dictionary, welches jedem Token eine Zahl zuweist

In [9]:
vocab = tokenizer.get_vocab()
first_20_items = {k: vocab[k] for k in list(vocab.keys())[:20]}
first_20_items

{'ĠArab': 4498,
 'ĠMinistry': 9475,
 'Ġpals': 39898,
 'Ġwriters': 8786,
 'Ġprison': 3770,
 'ĠRuby': 10888,
 'Ġscholarship': 18652,
 'Ġcardiovascular': 21134,
 'those': 25591,
 'above': 29370,
 'ĠGibson': 20400,
 'Ġraging': 30082,
 'assuming': 32935,
 'Ġ650': 22626,
 'Ġtaped': 36114,
 'umbled': 11137,
 'mg': 11296,
 'ĠPharmac': 37908,
 'Ġlith': 24491,
 'Ġmineral': 18352}

Es gibt insgesamt 50257 Tokens in diesem Tokenizer.

In [10]:
len(vocab)

50257

In [11]:
output = model(**encoded_input)
token_logits = output.logits

# Get the predicted token ids
predicted_token_ids = token_logits.argmax(dim=-1)
predicted_token_ids

tensor([[ 383,  290,  447,  247,   76,  407, 4336, 5887,   13]])

In [12]:
# Decode the tokens to text
predicted_text = [tokenizer.decode(t) for t in predicted_token_ids]

for index, t in enumerate(predicted_token_ids[0]):
  input_token = tokenizer.tokenize(text)[index]
  predicted_next_token = tokenizer.decode(t)
  print(input_token + " => " + predicted_next_token)


Hello =>  The
, =>  and
ĠI => �
âĢ => �
Ļ => m
m =>  not
Ġa =>  fan
Ġlanguage =>  expert
Ġmodel => .


In [16]:
# Get the top 5 predicted token ids
top_k = 5
top_k_values, top_k_indices = token_logits.topk(top_k, dim=-1)

# Iterate over the tokens and print the top 5 predictions for each one
for i in range(top_k_indices.shape[1]):
    input_token = tokenizer.tokenize(text)[i]
    print(f"Input {input_token}:")
    for j in range(top_k):
        token_id = top_k_indices[0, i, j].item()
        print(f"    {tokenizer.decode([token_id])}, score: {top_k_values[0, i, j].item()}")


Input Hello:
     The, score: -29.41666603088379
    ., score: -30.227386474609375
    ,, score: -30.28774642944336
     A, score: -30.631065368652344
    The, score: -30.689918518066406
Input ,:
     and, score: -60.01385498046875
     the, score: -60.604042053222656
     we, score: -60.68156814575195
     I, score: -60.799468994140625
     you, score: -60.957733154296875
Input ĠI:
    �, score: -69.6806869506836
     am, score: -70.5064926147461
    'm, score: -70.62347412109375
     have, score: -70.92636108398438
     think, score: -71.0676498413086
Input âĢ:
    �, score: -36.397064208984375
    �, score: -37.68267822265625
    �, score: -38.02922058105469
    �, score: -38.57345962524414
    �, score: -38.61290740966797
Input Ļ:
    m, score: -55.67150115966797
    ve, score: -57.00662612915039
    ll, score: -57.53662872314453
    d, score: -58.36833190917969
    �, score: -59.172340393066406
Input m:
     not, score: -60.79899215698242
     a, score: -61.2396240234375
     sorr

# Weitere Arten von Tasks
Auf folgender Seite findet ihr weitere mögliche Anwendungsfälle
https://huggingface.co/docs/transformers/main/en/quicktour#pipeline

Probiert gerne welche aus 🤗